In [ ]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

In [ ]:
data = pd.read_csv('ListingSecurityList.csv', error_bad_lines=False, sep = ';')

In [ ]:
eqs = data[data.s_sec_type_name_dop == 'Акции']

In [ ]:
url = start_url + 'AA'
html = urlopen(url)
bsObj = BeautifulSoup(html.read(), "lxml")

# print(bsObj.prettify())

In [ ]:
# for i, n in enumerate(bsObj.findAll("td")):
#     print(i)
#     print(n.text)

In [ ]:
#depricated
def get_data_google(url):
    result = {}
    html = urlopen(url)
    bsObj = BeautifulSoup(html.read(), "lxml")
    for snap in bsObj.find_all("table", {"class":"snap-data"}):
        for trs in snap.find_all("tr"):
            name = trs.find('td',{'class':'key'}).text
            name = re.sub('\n','',name)
            value = trs.find('td',{'class':'val'}).text
            value = re.sub('\n','',value)
            result[name] = value
        result['price'] = re.sub('\n','',bsObj.find('span',{'class':'pr'}).text)
        if bsObj.find('a',{'id':'sector'}):
            sector, industry = bsObj.find('div',{'class':'g-section g-tpl-50-50 g-split'}).div.text.split('>')
            result['sector'] = re.sub('\n','',sector)
            result['industry'] = re.sub('\n','',industry)
    return result

In [ ]:
# finance.yahoo.com
def get_data(url):
    result = {}
    html = urlopen(url)
    bsObj = BeautifulSoup(html.read(), "lxml")
    for i in range(0,32,2):
#         print(i)
#         print(bsObj.findAll("td")[i].text)
#         print(bsObj.findAll("td")[i+1].text)
        result[bsObj.findAll("td")[i].text] = bsObj.findAll("td")[i+1].text
    return result

In [ ]:
get_data(start_url + 'AA')

In [ ]:
# start_url = 'https://finance.google.com/finance?q='
start_url = 'https://finance.yahoo.com/quote/'

for s in eqs.s_RTS_code:
    print(s)
    url = start_url + s
    try:
        market_data = get_data(url)
#         print(market_data)
        for key,value in market_data.items():
            if key not in list(data.columns.values):
                data[key] = 0
            data.loc[data.s_RTS_code == s, key] = value
    except:
        print('bad')

In [ ]:
data.head()

In [ ]:
for name in data.columns[12:26]:
    data[name] = data[name].str.replace('.',',')

In [ ]:
data.to_csv('result_w_prices20191006.csv', sep=';')